**Пример развертывания с помощью docker базы данных postgres взаимодейсвующей с программой на python**

# Разные контейнеры

Програма в одном контейнере, база в другом. В теории так и должно быть, так что это самый правильный вариант.

Запуск контейнеров

In [ ]:
%%bash
docker run --rm -d\
    --name database\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    -p 5431:5432\
    custom_postgres

In [1]:
%%bash

# сеть
docker network create test_project_net
# postgres
docker run --rm -d\
    --name database\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    --net=test_project_net\
    custom_postgres
# python
docker run --rm -itd\
    --name python_psql\
    --net=test_project_net\
    python_psql

f0ffd8467c9308b627b33480b2819c9da7d4bf241aca09de90b4df0802cff7cf
0f3e07f8befd811abc0a837ecda944883a5dca9828184e01caaba892fb6f610e
ad4ae117eb86acd9b546d93ec0e622a803abe99b34434b1b557b3781179f3860


Надо создать таблицу с которой наша программа на python будет работать.

In [2]:
%%bash
# надо дождаться пока база данных
# развернется (но вообще это не обязательный шаг)
sleep 1
docker exec database \
    psql --username docker_app --dbname docker_app_db -f create_table.sql

CREATE TABLE


In [18]:
%%bash
docker exec python_psql python3 script.py

hello world


Удаление всего того, что насоздавали.

In [19]:
%%bash
docker stop database
docker stop python_psql
docker network rm test_project_net

database
python_psql
test_project_net


# Програма на хосте

В целях отладки программы бывает полезно поднять контейнер с базой а программу отлаживать на localhost. В данном разделе представлено именно такой случай.

Старт контейнера с базой и создание табилцы.

In [89]:
%%bash
# особое внимане порту - тут от принципиально важен
docker run --rm -d\
    --name database\
    -e POSTGRES_USER=docker_app\
    -e POSTGRES_PASSWORD=docker_app\
    -e POSTGRES_DB=docker_app_db\
    -p 5431:5432\
    custom_postgres

# надо дождаться пока база данных
# развернется (но вообще это не обязательный шаг)
sleep 1
docker exec database \
    psql --username docker_app --dbname docker_app_db -f create_table.sql

15991676e7aab8717c8293738eebb05dabf85fc58de3b253c6471f77a32d3131
CREATE TABLE


В python надо открыть connection с базой.

In [90]:
import psycopg2
import random
import string

conn = psycopg2.connect(
    port = "5431",# указываем порт
    dbname = "docker_app_db",
    user = "docker_app",
    password = "docker_app",
    host= "localhost"
)

Вставляем в новосозданную табличку 20 новый записей.

In [91]:
cur = conn.cursor()
for i in range(20):
    text = ''.join(random.choices(string.ascii_lowercase, k=20))
    query = f"INSERT INTO main_table (id, text) VALUES ('{i}', '{text}');"
    cur.execute(query)
cur.close()

И посмотрим, что это за записи.

In [92]:
cur = conn.cursor()
cur.execute("SELECT * FROM main_table;")
for i in cur:
    print(i)
cur.close()

('0', 'vhpcybtxzkurposczmmr')
('1', 'pvasvfadtvbdzpkdhwvn')
('2', 'rnsvqjmiitjgrddxwwck')
('3', 'gdskxulviyxqgtgnefcz')
('4', 'sojfgmbznllwyyrcalnw')
('5', 'rzcllpzwgcuicycxffbz')
('6', 'mniftnccdrqecyuvypmv')
('7', 'xzrastruqnadudpgyerf')
('8', 'oupgvmtlnbofodrpczne')
('9', 'krkqqponelbtfhupwvkj')
('10', 'xtffgiuzhvyjiehjakmp')
('11', 'cxikpqunjueeqppxwqsz')
('12', 'huhkcjxofpfuxlhxnhiw')
('13', 'eeblojrzwcfffehsaiwb')
('14', 'tmwprcmsmfjclqxbvuge')
('15', 'utbkrqkmnfdetqdazmvq')
('16', 'kfcowgauyirjetjlchrb')
('17', 'orztjpqzqxqerytovlze')
('18', 'wprfcnlztczggrwdsrkh')
('19', 'olbmbnbvchpszurgfvyx')


Отправляем изменения на базу и закрываем соединение.

In [93]:
conn.commit()
conn.close()

Для того, чтобы убедится, что все сработало, спрошу эту табличку напрямую из базы.

In [94]:
%%bash
docker exec database \
    psql --username docker_app --dbname docker_app_db -c 'SELECT * FROM main_table;'

 id |         text         
----+----------------------
 0  | vhpcybtxzkurposczmmr
 1  | pvasvfadtvbdzpkdhwvn
 2  | rnsvqjmiitjgrddxwwck
 3  | gdskxulviyxqgtgnefcz
 4  | sojfgmbznllwyyrcalnw
 5  | rzcllpzwgcuicycxffbz
 6  | mniftnccdrqecyuvypmv
 7  | xzrastruqnadudpgyerf
 8  | oupgvmtlnbofodrpczne
 9  | krkqqponelbtfhupwvkj
 10 | xtffgiuzhvyjiehjakmp
 11 | cxikpqunjueeqppxwqsz
 12 | huhkcjxofpfuxlhxnhiw
 13 | eeblojrzwcfffehsaiwb
 14 | tmwprcmsmfjclqxbvuge
 15 | utbkrqkmnfdetqdazmvq
 16 | kfcowgauyirjetjlchrb
 17 | orztjpqzqxqerytovlze
 18 | wprfcnlztczggrwdsrkh
 19 | olbmbnbvchpszurgfvyx
(20 rows)



Останавливаю созданный для этого примера контейнер.

In [88]:
%%bash
docker stop database

database
